# 0. Imports

In [1]:
import os
import sys
import numpy as np
%load_ext autoreload
%autoreload 2

# insert the directory where you extracted the zip file here:
git_dir = r"C:\Werk\Projecten\P1414_ROI\Cursus\StandAloneServiceZipfile"

sys.path.append(os.path.join(git_dir, 'Code'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox/Readers'))

from notebooks.background_scripting.v1.test_connection import TestConnection
test_connection = TestConnection()

Imports are succesfull!


# 1. Model specifications 

## 1.1. Select basemodel

In [2]:
from notebooks.background_scripting.v1.select_basemodel import ModelSettings

model_settings = ModelSettings()
model_settings.display_widgets()

Button(description='Update settings', layout=Layout(height='35px', width='99%'), style=ButtonStyle(button_colo…

Output()

'Model settings are:'

{
    "folder": "C:\\Werk\\Projecten\\P1414_ROI\\Cursus\\StandAloneServiceZipfile\\Model_database",
    "model": "model_cursus",
    "scenario_name": "run-test"
}


## 1.2. Model settings

In [3]:
import shutil
from notebooks.background_scripting.v1.modify_mdu import ModifyMDU, copy_model

model_path = copy_model(os.path.join(model_settings.settings['folder'], model_settings.settings['model']), model_settings.settings['scenario_name'])
modify_mdu = ModifyMDU(model_path)
modify_mdu.display_widgets()

Button(description='Update settings', layout=Layout(height='35px', width='99%'), style=ButtonStyle(button_colo…

Output()

'MDU settings are:'

{
    "tStart": 0.0,
    "tStop": 24.0,
    "mapInterval": 20.0,
    "DHYDRO location": "C:\\Program Files\\Deltares\\D-HYDRO Suite 2023.01 1D2D\\plugins\\DeltaShell.Dimr\\kernels\\x64\\dimr\\scripts\\run_dimr.bat"
}


## 1.3.1. Dambreak location manual

In [4]:
from notebooks.background_scripting.v1.modify_dambreak_v2 import DambreakWidget

use_template = False
dambreakWidget = DambreakWidget(model_path)
dambreakWidget.draw_map()
# loading the map takes a moment, wait until you see red dams on the map.

Map(center=[51.94553466305084, 4.658881452924391], controls=(ZoomControl(options=['position', 'zoom_in_text', …

HTML(value='<b style="color:black;font-size:18px;">Done! Inspect your results (4/4) </b>')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xdc\x00\x00\x03\x84\x08\x03\x00\x00\x00\x8c\xe6\…

## 1.3.2. Dambreak location from template

In [ ]:
from notebooks.background_scripting.v1.modify_dambreak_v2 import UseTemplateDambreak

use_template = True
dambreak_template = UseTemplateDambreak(model_path)
dambreak_template.draw_map()

## 1.4. Dambreak settings

In [5]:
from notebooks.background_scripting.v1.modify_dambreak_v2 import ModifyDambreak

if use_template == False:
    print("using the manually selected dike breach")
    add_dambreak = ModifyDambreak(model_path, dambreakWidget.settings, dambreakWidget.keringen, False, None)
else:
    print("using the dike breach from a template")
    add_dambreak = ModifyDambreak(model_path, None, None, True, dambreak_template.kering_choice)
add_dambreak.display_widgets()

Button(description='Update settings', layout=Layout(height='35px', width='99%'), style=ButtonStyle(button_colo…

Output()

'Dambreak settings are:'

{
    "crestLevelIni": -0.33,
    "t0": 0.0,
    "crestLevelMin": -2.0,
    "breachWidthIni": 5.0,
    "f1": 1.3,
    "f2": 0.04,
    "uCrit": 0.2
}


## 1.5. Boundary conditions

In [ ]:
from notebooks.background_scripting.v1.modify_boundaries import ModifyBoundaries, ModifyRhineDischarge

modify_boundaries = ModifyBoundaries(model_path, modify_mdu.settings, add_dambreak.dambreak_settings)
modify_boundaries.display_widgets()

# 2. Run Model

In [ ]:
from notebooks.background_scripting.v1.model_runner import ModelRunner

model_runner = ModelRunner(model_path)
model_runner.display_widgets()

# 3. Post processing

## 3.1. Plot map

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsMap, MapPlotter

output_path = os.path.join(model_path, 'dflowfm\output')
plot_settings = PlotSettingsMap(output_path)
plot_settings.display_widgets()

## 3.2. Plot dambreak

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsBreach, BreachPlotter

model_path = r"C:\Werk\Projecten\P1414_ROI\Cursus\StandAloneServiceZipfile\Model_runs\model_cursus_2023-05-06T10-08-54_tn-krimperwaard"
output_path = os.path.join(model_path, 'dflowfm\output')
plot_settings = PlotSettingsBreach(output_path)
plot_settings.display_widgets()

